In [1]:
import warnings
warnings. simplefilter(action='ignore', category=Warning)
import pandas as pd
import numpy as np
import pandas_ta as ta
import plotly.graph_objects as go
from plotly.subplots import make_subplots as ms
import backtesting
from backtesting import Strategy, Backtest
from tabulate import tabulate

Loading BokehJS ...

In [2]:
df4 = pd.read_csv('EURUSD_4Hour.csv')
dfd = pd.read_csv('EURUSD_1D.csv')

In [3]:
def drop_rows_where_zero_in_one_column(data, col_name):
    data = data[data[col_name]!=0]
    data = data.reset_index(drop=True)
    return data

In [4]:
df4 = drop_rows_where_zero_in_one_column(df4, 'Volume')

In [5]:
df4

,Gmt time,Open,High,Low,Close,Volume
0,01.01.2014 20:00:00.000,1.37553,1.37720,1.37390,1.37653,2724.8900
1,02.01.2014 00:00:00.000,1.37653,1.37746,1.37507,1.37660,7012.2300
2,02.01.2014 04:00:00.000,1.37661,1.37661,1.37250,1.37320,14175.5202
3,02.01.2014 08:00:00.000,1.37319,1.37421,1.36838,1.36892,50431.0206
4,02.01.2014 12:00:00.000,1.36892,1.36892,1.36294,1.36576,79223.8613
...,...,...,...,...,...,...
17214,06.09.2024 08:00:00.000,1.11157,1.11207,1.11026,1.11069,35319.1900
17215,06.09.2024 12:00:00.000,1.11070,1.11556,1.10656,1.10856,203661.3896
17216,06.09.2024 16:00:00.000,1.10854,1.10895,1.10695,1.10884,55519.9000
17217,06.09.2024 20:00:00.000,1.10886,1.10887,1.10822,1.10825,7471.4300


In [6]:
dfd = drop_rows_where_zero_in_one_column(dfd, 'Volume')

In [7]:
dfd

,Gmt time,Open,High,Low,Close,Volume
0,01.01.2014 00:00:00.000,1.37553,1.37720,1.37390,1.37653,2724.8899
1,02.01.2014 00:00:00.000,1.37653,1.37746,1.36294,1.36653,199117.5313
2,03.01.2014 00:00:00.000,1.36653,1.36719,1.35820,1.35874,193632.8438
3,05.01.2014 00:00:00.000,1.35935,1.36005,1.35890,1.35955,2823.0801
4,06.01.2014 00:00:00.000,1.35957,1.36527,1.35715,1.36296,189223.7344
...,...,...,...,...,...,...
3337,03.09.2024 00:00:00.000,1.10671,1.10702,1.10263,1.10485,254620.5300
3338,04.09.2024 00:00:00.000,1.10485,1.10951,1.10400,1.10801,213760.7301
3339,05.09.2024 00:00:00.000,1.10800,1.11197,1.10746,1.11110,235680.2400
3340,06.09.2024 00:00:00.000,1.11111,1.11556,1.10656,1.10825,353066.7396


In [8]:
dfd['ema5'] = ta.ema(close=dfd['Close'], length=5)
dfd['ema25'] = ta.ema(close=dfd['Close'], length=25)
dfd['ema30'] = ta.ema(close=dfd['Close'], length=30)
dfd[['macd', 'hist', 's_line']] = ta.macd(close=dfd['Close'], fast=12, slow=26, signal=9)

In [9]:
df4['ema5'] = ta.ema(close=df4['Close'], length=5)
df4['ema8'] = ta.ema(close=df4['Close'], length=8)
df4['ema25'] = ta.ema(close=df4['Close'], length=25)
df4['ema30'] = ta.ema(close=df4['Close'], length=30)
df4[['macd', 'hist', 's_line']] = ta.macd(close=df4['Close'], fast=12, slow=26, signal=9)

In [10]:
dfd.dropna(subset=['s_line'], inplace=True)
dfd.reset_index(drop=True, inplace=True)

In [11]:
df4.dropna(subset=['s_line'], inplace=True)
df4.reset_index(drop=True, inplace=True)

In [12]:
df4

,Gmt time,Open,High,Low,Close,Volume,ema5,ema8,ema25,ema30,macd,hist,s_line
0,09.01.2014 04:00:00.000,1.35791,1.36013,1.35750,1.36010,18944.9105,1.359058,1.359408,1.361763,1.362401,-0.002102,0.000120,-0.002222
1,09.01.2014 08:00:00.000,1.36009,1.36217,1.35931,1.36017,49411.8106,1.359429,1.359577,1.361640,1.362257,-0.001915,0.000245,-0.002161
2,09.01.2014 12:00:00.000,1.36010,1.36330,1.35480,1.35850,90896.0508,1.359119,1.359338,1.361399,1.362015,-0.001880,0.000224,-0.002105
3,09.01.2014 16:00:00.000,1.35849,1.35954,1.35809,1.35901,29326.6406,1.359083,1.359265,1.361215,1.361821,-0.001790,0.000251,-0.002042
4,09.01.2014 20:00:00.000,1.35901,1.36106,1.35876,1.36062,17111.0901,1.359595,1.359566,1.361169,1.361743,-0.001571,0.000376,-0.001948
...,...,...,...,...,...,...,...,...,...,...,...,...,...
17181,06.09.2024 08:00:00.000,1.11157,1.11207,1.11026,1.11069,35319.1900,1.110771,1.110252,1.108676,1.108637,0.000935,0.000612,0.000323
17182,06.09.2024 12:00:00.000,1.11070,1.11556,1.10656,1.10856,203661.3896,1.110034,1.109876,1.108667,1.108632,0.000783,0.000368,0.000415
17183,06.09.2024 16:00:00.000,1.10854,1.10895,1.10695,1.10884,55519.9000,1.109636,1.109646,1.108680,1.108645,0.000677,0.000210,0.000467
17184,06.09.2024 20:00:00.000,1.10886,1.10887,1.10822,1.10825,7471.4300,1.109174,1.109335,1.108647,1.108620,0.000540,0.000058,0.000482


In [13]:
dfd

,Gmt time,Open,High,Low,Close,Volume,ema5,ema25,ema30,macd,hist,s_line
0,09.02.2014 00:00:00.000,1.36165,1.36269,1.36160,1.36186,3053.4099,1.359239,1.359148,1.360541,-0.001486,0.001116,-0.002602
1,10.02.2014 00:00:00.000,1.36185,1.36514,1.36184,1.36451,143303.6250,1.360996,1.359561,1.360797,-0.000846,0.001404,-0.002251
2,11.02.2014 00:00:00.000,1.36451,1.36829,1.36295,1.36366,182680.5000,1.361884,1.359876,1.360981,-0.000403,0.001478,-0.001881
3,12.02.2014 00:00:00.000,1.36366,1.36527,1.35623,1.35877,197226.8281,1.360846,1.359791,1.360839,-0.000442,0.001151,-0.001593
4,13.02.2014 00:00:00.000,1.35876,1.36918,1.35855,1.36759,202744.2031,1.363094,1.360391,1.361274,0.000236,0.001464,-0.001227
...,...,...,...,...,...,...,...,...,...,...,...,...
3304,03.09.2024 00:00:00.000,1.10671,1.10702,1.10263,1.10485,254620.5300,1.106713,1.104511,1.102968,0.003769,-0.001709,0.005477
3305,04.09.2024 00:00:00.000,1.10485,1.10951,1.10400,1.10801,213760.7301,1.107146,1.104780,1.103294,0.003493,-0.001587,0.005081
3306,05.09.2024 00:00:00.000,1.10800,1.11197,1.10746,1.11110,235680.2400,1.108464,1.105266,1.103797,0.003484,-0.001277,0.004761
3307,06.09.2024 00:00:00.000,1.11111,1.11556,1.10656,1.10825,353066.7396,1.108392,1.105496,1.104085,0.003209,-0.001241,0.004451


In [14]:
def l_ema_and_h_ema(data):
    l_ema = np.full(len(data), 0)
    h_ema = np.full(len(data), 0)
    for i in range(len(data)):
        if data['ema25'][i] < data['ema30'][i]:
            l_ema[i] = data['ema25'][i]
            h_ema[i] = data['ema30'][i]
        elif data['ema30'][i] < data['ema25'][i]:
            l_ema[i] = data['ema30'][i]
            h_ema[i] = data['ema25'][i]
        else:
            l_ema[i] = data['ema30'][i]
            h_ema[i] = data['ema25'][i]
    return l_ema, h_ema

In [15]:
l_ema , h_ema = l_ema_and_h_ema(dfd)
dfd['l_ema'] = l_ema
dfd['h_ema'] = h_ema

In [16]:
l_ema , h_ema = l_ema_and_h_ema(df4)
df4['l_ema'] = l_ema
df4['h_ema'] = h_ema

In [21]:
def get_buy_condition(data):
    buy = np.full(len(data), np.nan)
    for index1, row1 in data.iterrows():
        if row1.macd < 0 and row1.s_line < 0 and row1.macd > row1.s_line:
            for index2, row2 in data[index1:].iterrows():
                if row2.ema5 < row2.h_ema:
                    for index3, row3 in data[index2:].iterrows():
                        if row3.ema5 > row3.h_ema and row3.macd > row3.s_line:
                            buy[index3] = True
                            # print(index3)
    buy = pd.Series(buy).ffill()
    return buy

In [22]:
def get_sell_condition(data):
    sell = np.full(len(data), np.nan)
    for index1, row1 in data.iterrows():
        if row1.macd > 0 and row1.s_line > 0 and row1.macd < row1.s_line:
            for index2, row2 in data[index1:].iterrows():
                if row2.ema5 > row2.l_ema:
                    for index3, row3 in data[index2:].iterrows():
                        if row3.ema5 < row3.l_ema  and row3.macd < row3.s_line:
                            row.sell = True
                            # print(index3)
    buy = pd.Series(sell).ffill()
    return sell

In [23]:
dfd['buy'] = get_buy_condition(dfd)

In [ ]:
dfd['sell'] = get_sell_condition(dfd)